Подключаем необходимые для работы библиотеки

In [1]:
import numpy as np
import copy

Настройка форматы вывода

In [2]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

Напишем функцию для решения линейных систем уравнений. Предпологаем, что $A$ невырожденная квадратная матрица размера $n \; x \; n$.

In [3]:
def solve_linear_system(mtx, b):
    return np.linalg.inv(mtx) @ b

### Симплекс таблицы

Для реализации **симплекс метода** нам необходимо воспользоваться симплекс-таблицами, далее будет приведен **симплекс метод**, который использует симплекс-таблицы в ходе работы.

Введем некоторые необходимые для нас понятия и напомним некоторые определения.

#### Формы задач линейного программирования

Пускай мы имеем задачу в виде

$$ z(x) = \langle c, x \rangle \rightarrow max $$ $$ Ax \leq b $$ $$ x \geq 0 $$


Такую форму задачи мы будем называть **стандартной**.

Так же нам предстаить иметь дело с **канонической** формой задачи, они имеет следующий вид: 
$$ z(x) = \langle c, x \rangle \rightarrow max $$ 
$$ Ax = b $$ 
$$ x \geq 0 $$

Утверждается, что стандартная и каноническая форма задач являетсю эквивалентными.

#### Базисный план задачи линейного программирования и его базис

Рассмотрим матрицу $A$:

\begin{equation*}
\left(
\begin{array}{cccc}
a_{11} & a_{12} & \ldots & a_{1n}\\
a_{21} & a_{22} & \ldots & a_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
a_{m1} & a_{m2} & \ldots & a_{mn}
\end{array}
\right)
\end{equation*}

Требуем, что 
$ rank(A) = m $
, так же  $ m \leq n $, то есть мы требуем линейную независимость строк матрицы $A$.

Далее считаем, что $ b \neq 0 $.

Рассмотрим задачу линейного программирования в канонической форме:
$$ z(x) = \langle c, x \rangle \rightarrow max $$ $$ Ax = b $$ $$ x \geq 0 $$

Множество $ \Omega \subset \mathbb{R}^n$ вида $$ \Omega := \{ x \subset \mathbb{R}^n : Ax = b, x \geq 0\} $$

будем называть множеством ***допустимых планов*** задачи линейного программирования, или ***полиэдром*** планов этой задачи. Элементы множества $\Omega$ называются ***допустимыми планами*** задачи.

***Допустимые планы***, на которых целевая функция $z(x)$ принимает максимальное значение, называются ***оптимальными планами***.

***Допустимый план*** $ x = (x_1, x_2,\dots, x_n)^T $ называется ***базисным планом*** задачи, если у него не менее $ (n - m) $ нулевых компонет, а остольные $ m $ компонент соответствуют линейной независим столбцам матрицы $ A $, обозначим эти компоненты как $$ x_{j1}, x_{j2}, \dots, x_{jm}, $$ соответствующие этим компонентам столбцы матрицы $ A $ обозначим $$ A_{j1}, A_{j2}, \dots, A_{jm} $$

Набор этих слобцов называется ***базисом*** данного базисного плана. Если у базисного плана $m$ положительных компонент, $x_{j1} \geq 0, \dots, x_{jm} \geq 0$, то он называется ***невырожденным***, в противном случае,- ***вырожденным***.

Положим $ J = \{ j_1, \dots, j_m \} $ - множество индексов базисного плана.

Заметим, что мы вводили это определения не просто так:). Известно, что если задача линейного программирования является разрешимой, то по крайней мере, один ***оптимальный план*** будет её базисным. 

Если, полиэдр $\Omega$ не пуст, то у него имеется, по крайней мере, одна вершина, кроме того как минимум одна вершина полиэдра соответствует ***базисному плану***, который к тому же является ***оптимальным***.

Геометрическая интерпретация базисных планов - вершины полиэдра.

Стоит уточнить, что мы так же можем встретиться с полиэдральными множествами, то есть не ограниченными полиэдрами. В таких ситуациях может не существовать решения задачи линейго программирования.

Теперь введем понятие ***симплекс-таблицы***.

Пусть система столбцов $ \{ A_j \subset \mathbb{R}^n: j \in J \} $, является базисом некоторого базисного плана $x$. Ясно также, что эти столбцы образуют базис в пространстве $\mathbb{R}^m$ (их m штук, и по определению они являются линейно независимыми). Следовательно, все столбцы матрицы $A$ можно однозначно представить в виде линейной комбинации базисных столбцов, то есть

$$ A_k = \sum_{j \in J}\lambda_{jk}A_j, \; \forall k \in \{1, 2, \dots, n\}$$
Числа $\lambda_{jk}$, входящие в разложение, принято называть ***коэффициентами замещения***.
   
***Оценками замещения*** назовём следующие величины

$$\delta_k = \sum_{j \in J}c_j\lambda_{jk} - c_k, \forall k \in \{ 1, 2, \dots, n \} $$
$$ \delta = (\delta_1, \dots, \delta_n) $$

Для ***симплекс-метод*** нам достаточно иметь информацию про коэффициенты замещения, оценки замещения и текущий базисный план. Мы будем проводить релаксации анализирую текущую оценки с целью улучшить значение целевой функции (в некотором из смыслов уменьшить / увеличить).

Как уже можно было заметить, получить ***коэффициенты замещения*** и ***оценки замещения*** не составит для нас особой трудности, проблемы возникают при поиски базисного плана, про это мы поговорим позже, а сейчас разберемся как находить ***коэффициенты замещения***, и следовательно ***оценки замещения***.

Посмотри на нашу систему столбцов матрицы $A$:
$$ \{ A_j \subset \mathbb{R}^n: j \in J \} $$
Теперь состовим из нее матрицу $T$, следующим образом:
$$ T = (A_{j1}, A_{j2}, ..., A_{jm}), $$ 
где $ A_{i} $ - соответствующие столбцы матрицы $ A, i \in \{j1, j2, \dots, jm\} = J $

Так как наша система столбцов по определению является независимой, столбцов мы имеем $m$ штук, и высота каждого столбца $m$, то выходит, что матрицы T является квадратной невырожденной матрицой размера $m \; x \; m$

Тогда запись,
$$ A_k = \sum_{j \in J}\lambda_{jk}A_j \; \Longleftrightarrow T\Lambda_k = A_k \; \forall k \in \{1, 2, \dots, n\},$$ где $$ \Lambda_k = ( \lambda_{jk} | j \in J ),$$ вектор - где $i$-ая компонета является коэффициентом замещения в разложении столбца $A_k$ по нашому базису.


То есть задача поиска коэффициентов замещения, сводит к решению $n$ систем линейных уравнений. Зная коэффициенты замещения мы без проблем по формуле вычисляем оценки заемщения.

Параллельно с описанием симплекс-метода будем считать его асимптотику. Так как мы решаем $n$ систем линейных уравнений, с матрицей размера $m x m$ (считаем, что мы можем решить слау за $O(n^3)$, где n - размер матрицы), получаем затраты ***$O(nm^3)$*** для нахождения коэффициентов замещения. Для нахождения оценок замещения мы затратим ***$O(nm)$*** .Пока имеем следующую асимптотику: $$ O(nm^3 + nm) = O(nm^3), $$ где m - количество строк матрицы $A$, $n$ - соответсвенно количество столбцов.

Раскроем еще немного подробностей про симплекс метод. Так как было сказано выше этот алгоритм является итеративным,
то есть на каждом шаге мы проводим релаксацию с целью улучшить значение нашей функции. То есть мы проводим некоторый
анализ коэффициентов замещения, оценок замещания и текущего базисного плана, что бы принять некторое решение.
Каждый базисный план соответствует некоторой вершине полиэдра, и на каждом шаге мы пытаемся перейти по некотору ребру полиэдра (или полиэдрально множества) с целью попасть в вершину, где значение нашей функции будет лучше.


С целью классификации положений приведем следующие разбиения, которые дадут нам дизъюнктивное разбиений нашего
пространства состояний на три множества.

$a.)$ $\delta_k \geq 0, \forall k \in \{1, 2, \dots, n \} $ (все оценки замещения являются неотрицательными)

$b.)$ $\exists s \notin J: \delta_s < 0, \forall j \in J, \lambda_{js} \leq 0$ (существует отрицательная оценка замещения, причём коэффициенты разложения соответствующего столбца по базису неположительные)

$c.)$ $\exists s \notin J: \delta_s < 0, \exists j \in J: \lambda_{js} > 0$ (существует отрицательный коэффициент замещения, причём среди коэффициентов разложения соответсвующего столбца по базису есть положительный)

Заметим, что для любых значений параметров $\delta_k$ и $\lambda_{jk}$ выполняется одно и только одно из условний **a.), b.) и c.)**. Каждому из этих условий соответсвует одно из следующих правил симплекс метода.

**Теорема 1** (первая теорема Данцига, правило оптимальности). Если выполнено условие **a.)**, то базисный план $x$ является оптимальным, то есть решением нашей задачи.

**Теорема 2** (вторая теорема Данцига, правило отсутствия решения). Если выполняется условие b.), то задача линейного программирования не имеет решения.

Стоит заметить, что решение мы можем не находить в двух случаях, если система неравенств является противоричивой,
то полиэдр является пустым множеством, или в случае, если мы имеем полиэдральное множество и не можем достигнуть min/max. 

Вторая теорема Данцига соответствует случаю полиэдрального множества и когда функция на нем может принимать бесконечно большие значения. 

Ясно, что если выполняется условие **a.)** или **b.)**, то решение на этом заканчивается. В том случае, если выполнено
условие **c.)** мы еще не достигли оптимального базисного решение и следует продолжать релаксации, насчет этого поговорим
немного позже, и там же приведем третью теорему Данцига.

Осталось последнее, определеним ***симплекс-таблицу***. Таблица строиться по текущему базисному плану $x$ и его базису $A_j, j \in J.$ Таблица имеет следующий вид:

\begin{array}{|c|c|}
\hline
  T_0 & A_1 & A_2 & \dots & A_n & x \\\hline
  A_{j1} & \lambda_{j_11} & \lambda_{j_12} & \dots & \lambda_{j_1n} & x_{j_1} \\\hline
  A_{j2} & \lambda_{j_21} & \lambda_{j_22} & \dots & \lambda_{j_2n} & x_{j_2} \\\hline
  \dots  & \dots & \dots & \dots & \dots & \dots \\\hline
  A_{jm} & \lambda_{j_m1} & \lambda_{j_m2} & \dots & \lambda_{j_mn} & x_{j_m} \\\hline
  \delta & \delta_1 & \delta_2 & \dots & \delta_n & \langle c, x \rangle \\\hline
\end{array}


Данные таблицы будут использовать для симплекс метода, ниже приводом реализацию функции которая генерирует симплекс
таблицу из данных.

In [4]:
def create_simplex_tableau(mtx, b, c, basis, x0):
    m = mtx.shape[0]
    n = mtx.shape[1]
    
    basis_matrix = np.hstack([mtx[:, idx][:, np.newaxis] for idx in basis])
    
    simplex_tableu = np.hstack([solve_linear_system(basis_matrix, mtx[:, idx])[:, np.newaxis] 
                                for idx in range(mtx.shape[1])])
    
    last_row = np.array([ sum(simplex_tableu[i][k] * c[basis[i]] for i in range(m)) - c[k] for k in range(n) ])
    simplex_tableu = np.vstack([simplex_tableu, last_row])
    
    last_column = np.vstack([np.array([x0[basis[i]] for i in range(m)])[:, np.newaxis], np.dot(c, x0)])
    simplex_tableu = np.hstack([simplex_tableu, last_column])
    
    return simplex_tableu

Реализуем функцию, которая проверяет условие ***a.)*** (соответствует первой теореме Данцига), то есть, что используя данную симплекс таблицу мы достигли оптимального решения.

In [5]:
def optimal_solve(simplex_tableu):
    m = simplex_tableu.shape[0]
    n = simplex_tableu.shape[1]
    
    lambdas = simplex_tableu[m - 1]
    
    optimal = True
    for k in range(n - 1):
        if lambdas[k] < 0:
            optimal = False
    
    return optimal

Реализуем функцию, которая проверяет условие ***b.)*** (соответствует второй теореме Данцига), то есть, что используя данную
симплекс таблицу, мы не можем получить оптимальное решение. (задача является неограниченной)

In [6]:
def unbounded_problem(simplex_tableu, basis):
    m = simplex_tableu.shape[0]
    n = simplex_tableu.shape[1]
    
    lambdas = simplex_tableu[m - 1]
    
    for i in range(n - 1):
        if i not in basis and lambdas[i] < 0:
            ok = True
            for j in range(0, m - 1):
                if simplex_tableu[j][i] > 0:
                    ok = False
            if ok:
                return True
    
    return False

### Cимплекс метод

Теперь когда мы разобрались в том, что такое симплекс-таблицы, научились их строить и  рассмотрели разбиение пространства состояний, опишем ***симплекс метод***

***Симплекс метод*** позволяет получить решение задачи линейного программирования в результатет итерационного процесса - ряда однотипных повторяющихся вычислений. Каждый последующий шаг итерационного процесса дает решение, более близкое к оптимальному решению задачи, нежели предыдущий шаг. В
течение такого процесса происходит целенаправленный перебор базисных
планов полиэдра в поисках оптимального, причём значение целевой
функции на каждом следующем шаге, по крайней мере, не убывает.
Поскольку базисные планы являются, с геометрической точки зрения,
вершинами полиэдра, причём решением задачи, если оно существует,
обязательно будет некоторый базисный план, то происходит движение «по
вершинам» полиэдра к оптимальной вершине.

Таким образом, симплекс-метод по заданному исходному базисному
плану $x_0$ генерирует последовательность базисных планов $x_1 , x_2, \dots, x_p$
вместе с их базисами. Развитие процесса может происходить в двух
направлениях: либо изменяется базисный план и его базис (в этом случае
значение целевой функции увеличивается), либо базисный план
(вырожденный) остаётся прежним, а меняется только его базис (в этом
случае значение целевой функции увеличивается).

Пусть мы имеем задачу в нормальной форме
$$ z(x) = \langle c, x \rangle \rightarrow max $$
$$ Ax = b $$
$$ x \geq 0 $$
$$ m < n, \; rankA = m, b \neq 0 $$

Задачу имеет смысл решать, если для базисного плана $x$ и его базиса $\{ A_j : j \in J \}$ выполняется условие ***c.)*** (то есть в силу второй теоремы Данцига мы еще не достигли оптимального базисного плана). В этом случае нужно переходить к новому базису $\{A_{j}^{'}: j \in J' \}$.

Итак, пусть $s \notin J$ и $\delta_s < $; находим индекс $r \in J$ из условия
$$ \frac{x_r}{\lambda_{rs}} = \min\limits_{ j \in J, \lambda{js}>0}\frac{x_j}{\lambda{js}} $$
Положим
$$ \varepsilon = \frac{x_r}{\lambda_{rs}}, $$ имеем, что $\varepsilon \geq 0.$

Новое множество индексов $J'$ формируется из множества $J$ следующим образом. Индекс $r$ исключаем из множества $J$, а индекс, $s$, наоборот, включаем во множество $J'$. Итак, $$ J' = \{J \cup \{s\}\} /\ \{r\}. $$

Положим
\begin{equation*}
x_j^{'} = 
 \begin{cases}
   x_j - \frac{\lambda_{js}}{\lambda_{rs}}x_{r} & \text{$j \in J /\ \{r\}$}\\
   \frac{\lambda_{js}}{\lambda{rs}}x_{r} &\text{$j = s$}\\
   0  & \text{$j \notin J'$}
 \end{cases}
\end{equation*}

Теперь можем сформулировать третью теорему Данцига.

***Теорема 3.*** (третья теорема Данцига, правило перехода к новому базисному плану). Если выполняется условие ***c.)***, то вектор $x^{'} \in \mathbb{R}^n$, компоненты которого определены были выше, является базисным планом поставленной задачи, а множество векторов $\{A_j^{'}: j \in J^{'}\}$ является его базисом, причем
$$ z(x^{'})=z(x)-\varepsilon x_{s}$$.

При этом говорят, что столбец $A_{r}$ ***выводится из базиса***, а столбец $A_{s}$ ***вводится*** в базис. Элемент $\lambda_{rs}$ называется ***ведущим*** и помечается в симплекс таблицы кружочком, для проведение дальнейших операций с ним.

Мы не будем приводить явные формулы пересчета симплекс таблиц при переходе от одного базиса к другому, но утверждается, что они эквиваленты тому, что мы пересчитываем симплекс таблицу так, что бы ведещий элемент новой симлекс таблицы стал равен 1, а все остальные элементы ведущего столбца - равными 0. Мы можем добиться этого использую элементарные преобразования симплекс таблицы. 

Ниже реализуем описанный выше симплекс метод, который работает с симплекс таблицей при известном базисном плане. Предполагается, что решение сущестует.

In [7]:
def solve_simplex_tableu(tableu, basis, log = None):
    tableu = copy.deepcopy(tableu)
    basis = copy.deepcopy(basis)
    
    m = tableu.shape[0]
    n = tableu.shape[1]
        
    def find_optimal_i_j(tableu):
        optimal_j = 0
        lambdas = tableu[m - 1]
        
        for j in range(n-1):
            if lambdas[j] < 0:
                optimal_j = j
                break
        
        optimal_i = 0
        min_value = 1e9

        for i in range(0, m-1):
            x_i = tableu[i][n - 1]
            coeff = tableu[i][optimal_j]
            if coeff > 0:
                if x_i / coeff < min_value:
                    min_value = x_i / coeff
                    optimal_i = i
                    
        return (optimal_i, optimal_j)
        
    def gauss_iterate(idx1, idx2, tableu):
        tableu[idx1] /= tableu[idx1][idx2]
        for k in range(0, m):
            if k != idx1:
                tableu[k] -= tableu[idx1] * tableu[k][idx2]
    
    if unbounded_problem(tableu, basis):
        raise Exception('No solution')
    
    new_basis = basis
    num_of_iteration = 1
    while not optimal_solve(tableu):
        if log:
            print(f'Iteration={num_of_iteration}')
            print()
            num_of_iteration += 1
            print(tableu)
        result = find_optimal_i_j(tableu)
        new_basis[result[0]] = result[1] 
        gauss_iterate(result[0], result[1], tableu)
    
    if log:
        print(f'Iteration={num_of_iteration}')
        print()
        num_of_iteration += 1
        print(tableu)
    
    answer = np.array([0] * (n - 1), dtype=np.float32)
    for i in range(m-1):
        answer[new_basis[i]] = tableu[i][n - 1]
    
    return answer

Реализуем вспомогательную функцию, которая из входных данных делать симплекс таблицу и ищет решаем задачи линейнго
программирования.

Функция работает с задачей в нормальной форме, то есть:
$$ z(x) = \langle c, x \rangle \rightarrow max $$
$$ Ax = b $$
$$ x \geq 0 $$

In [8]:
def solve_with_basis(A, b, c, basis, x0, log = False):
    simplex_tableau = create_simplex_tableau(A, b, c, basis, x0)
    return solve_simplex_tableu(simplex_tableau, basis, log)

Теперь когда мы уже научились решать задачу при известном базисном плане, остается вопрос, что делать, если базисный
план не известный ? В этом случае нам поможет метод исскуственного базиса.

Рассмотрим каноническую задачу линейного программирования

$$ z(x) = \langle c, x \rangle \rightarrow max $$
$$ Ax = b $$
$$ x \geq 0 $$

Не теряя общности, считаем, что $$ b \geq 0$$
Обозначи
$$ u = (u_1, \dots, u_m)^T, U = (x_1, \dots, x_n, u_1, \dots, u_m)^T, c = (\underbrace{0, \dots, 0}_{n}, \underbrace{-1, \dots, -1}_{m})^T $$

Запишем вспомогательную задачу:
$$ F(x) = \langle C, U \rangle = \sum_{i=1}^{m}(-u_i) \rightarrow max $$
$$ Ax + u = b$$
$$ x \geq 0,\; u \geq 0,\; b \geq 0 $$
Или иначе:
$$ -(u_1 + u_2 + \dots + u_m) \rightarrow max $$
$$ a_{11}x_1 + \dots + a_{1n}x_n + u_1 = b_1, $$
$$ \dots $$
$$ a_{m1}x_{1} + \dots + a_{mn}{n}x_{n} + u_{m} = b_m $$
$$ x \geq 0, b \geq 0, u \geq 0 $$

Рассмотри едичные векторы $e_1 = (1, 0, \dots, 0)^T, e_2 = (0, 1, 0, \dots, 0)^T, \dots, e_m = (0, 0, \dots, 0, 1)^T $. Вектор $U^{0}=(\underbrace{0,\dots,0}_{n},b_1, \dots, b_m)^T $ будет базисным планом вспомогательной задачи с базисом $\{ e_1, e_2, ..., e_m $\}

Вспомогательная задача имеет решение, так как целевая функция является линейной и ограниченной на допустимом множестве (а как так $U^{0}$ подходит под ограничение вспомогательной задачи, то допустимое множество не пусто).

Так как мы раньше научились решать симплекс методом задачи линейного программирования с известным начальным планом, мы можем решить вспомогательную задачу, найдя оптимальный базисный план этой задачи $U^{*}=(x^{*}, u^{*})^T \in \mathbb{R}^{n + m} $

Далее утверждается следующие, если $u^{*} \neq 0$, то допустимное множество базисных планов пусто. Если $u^{*}=0,$ то $x^{*}$ является базисным планом исходной задачи.

### Двухфазный симплекс метод

Теперь ***двухфазный симплекс метод*** заключается в следующем, так как нам изначально не известней базисный план, на первом этапе ищем базисный план с помощью метода исскуственного базиса, на втором этапе решаем изначальную задачу, уже с найденным базисным планом. 

Сначала реализуем функцию для поиска базисного плана. На этом этапе сделаем проверку на непустоту множества допустимых планов.

In [9]:
def first_phase(A, b, c, log = False):
    # Ax = b, (c, x) -> max
    # Find basis
    # Add V1, V2, ..., Vm
    EPS = 1e-6
    
    m = A.shape[0]
    n = A.shape[1]
    
    A = np.hstack([np.eye(m), A])
    new_c = np.hstack([-np.ones(m), [0] * n])
    
    basis = np.arange(m)
    x0 = np.hstack([b, np.zeros(n)])
    
    print('First phase')
    x_result = solve_with_basis(A, b, new_c, basis, x0, log)
    
    func_value = np.dot(new_c, x_result)
    if (abs(func_value) > EPS):
        raise Expection('No solution')
    
    return x_result[m: ]

In [10]:
def second_phase(A, b, c, log = False):
    # Ax = b, (c, x) -> max
    EPS = 1e-6

    
    x0 = first_phase(A, b, c, log)
        
    basis = []
    for idx, value in enumerate(x0):
        if abs(value) > EPS:
            basis.append(idx)
    
    while len(basis) < A.shape[0]:
        for idx in range(A.shape[0]):
            if idx not in basis:
                basis.append(idx)
    
    print('Second phase')
    answer = solve_with_basis(A, b, c, basis, x0, log = True)
    
    return answer 

Напишем функций, которая решает задачу линейного программирования в стандартной форме c помощью двухфазного симплекс метода, то есть
$$ \langle c, x \rangle \rightarrow max $$
$$ Ax \leq b $$
$$ x \geq 0 $$
$$ rankA = m < n $$

In [73]:
def solve(A, b, c):
    # Ax = b, (c, x) -> max
    A = copy.deepcopy(A)
    b = copy.deepcopy(b)
    c = copy.deepcopy(c)
    
    m = A.shape[0]
    n = A.shape[1]
    
    A = np.hstack([np.eye(m), A])
    c = np.hstack([np.zeros(m), c])
    
    for i in range(m):
        if b[i] < 0:
            b[i] *= -1
            A[i] *= -1
    
    try:
        result_x = second_phase(A, b, c, log=True)
    except:
        return None
    
    return result_x[m:]

Теперь когда все готово, продемострируем двухфазный симплекс метод, решив задачу из 3 варианта. Так же выведим симплекс таблицы для первой и второй фазы, формат таблиц описан в соответствующем разделе.

In [74]:
def test():
    from scipy.optimize import linprog
    A = np.array([
        [1, 2, 2],
        [5, -1, 0],
        [-8, 2, 0],
        [-1, -3, 1],
        [1, 1, -2]
    ], dtype=np.float32)
    
    b = np.array([
        40, 30, 1, 0, -1
    ])
    
    c = np.array([
        2, 2, 1
    ])
    
    result = solve(A, b, c)
    if result is None:
        print('No solution')
        return 
    
    print('x*', result)
    print('func', np.dot(c, result))

In [75]:
test()

First phase
Iteration=1

[[  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.   2.   2.  40.]
 [  0.   1.   0.   0.   0.   0.   1.   0.   0.   0.   5.  -1.   0.  30.]
 [  0.   0.   1.   0.   0.   0.   0.   1.   0.   0.  -8.   2.   0.   1.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   1.   0.  -1.  -3.   1.   0.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.  -1.  -1.  -1.   2.   1.]
 [  0.   0.   0.   0.   0.  -1.  -1.  -1.  -1.   1.   4.   1.  -5. -72.]]
Iteration=2

[[  1.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.   2.   2.  40.]
 [  0.   1.   0.   0.   0.   0.   1.   0.   0.   0.   5.  -1.   0.  30.]
 [  0.   0.   1.   0.   0.   0.   0.   1.   0.   0.  -8.   2.   0.   1.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   1.   0.  -1.  -3.   1.   0.]
 [  0.   0.   0.   0.   1.   0.   0.   0.   0.  -1.  -1.  -1.   2.   1.]
 [  1.   0.   0.   0.   0.   0.  -1.  -1.  -1.   1.   5.   3.  -3. -32.]]
Iteration=3

[[ 1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.  2.  2. 40.]
 [ 0.  1.  0

Посмотри на пример 1:
$$ x + y \rightarrow min $$
$$ x \geq 1 $$
$$ y \geq 0 $$
Запишем задачу в ***стандартной форме***:
$$ -x - y \rightarrow max $$
$$ -x \leq -1 $$
$$ y \geq 0 $$

In [78]:
def test():
    A = np.array([
        [-1, 0]
    ])
    c = np.array([-1, -1], dtype=np.float32)
    b = np.array([-1], dtype=np.float32)
    
    result = solve(A, b, c)
    print('x_res= ', result)
    print('func(x_res)= ', np.dot(c, result))

In [79]:
test()

First phase
Iteration=1

[[ 1. -1.  1.  0.  1.]
 [ 0.  1. -1.  0. -1.]]
Iteration=2

[[ 1. -1.  1.  0.  1.]
 [ 1.  0.  0.  0.  0.]]
Second phase
Iteration=1

[[-1.  1.  0.  1.]
 [ 1.  0.  1. -1.]]
x_res=  [1. 0.]
func(x_res)=  -1.0


Возращаемся к исходной задаче (меняем знак значения функции). Как видим решение задачи из примера 1 является вектор $ x^{*} = (1, 0)^T $ на котором функция принимает значение
$f(x^{*}) = 1 $

Далее рассмотрим пример 2:
$$ x + y \rightarrow min $$
$$ 2x + y \geq 1 $$
$$ x, y \geq 0 $$
Запишем задачу в стандратной форме:
$$ - x - y \rightarrow min $$
$$ -2x - y \leq -1 $$
$$ x, y \geq 0 $$


In [82]:
def test1():
    A = np.array([
        [-2, -1]
    ])
    c = np.array([-1, -1], dtype=np.float32)
    b = np.array([-1], dtype=np.float32)
    
    result = solve(A, b, c)
    print('x_res= ', result)
    print('func(x_res)= ', np.dot(c, result))

In [83]:
test1()

First phase
Iteration=1

[[ 1. -1.  2.  1.  1.]
 [ 0.  1. -2. -1. -1.]]
Iteration=2

[[ 0.5 -0.5  1.   0.5  0.5]
 [ 1.   0.   0.   0.   0. ]]
Second phase
Iteration=1

[[-0.5  1.   0.5  0.5]
 [ 0.5  0.   0.5 -0.5]]
x_res=  [0.5 0. ]
func(x_res)=  -0.5


Меняем знак значения функции, имеем решение в точке $ x^{*} = (0.5, 0)^T, f(x^{*}) = 0.5 $

Теперь разберемся с пример 3:
$$ 2x + y \rightarrow min $$
$$ x + y \geq 1 $$
$$ x, y \geq 0 $$
Аналогично, переходим к ***стандартной форме***
$$ -2x - y \rightarrow max $$
$$ -x - y \leq -1 $$
$$ x, y \geq 0 $$

In [84]:
def test2():
    A = np.array([
        [-1, -1]
    ])
    c = np.array([-2, -1], dtype=np.float32)
    b = np.array([-1], dtype=np.float32)
    
    result = solve(A, b, c)
    print('x_res= ', result)
    print('func(x_res)= ', np.dot(c, result))

In [85]:
test2()

First phase
Iteration=1

[[ 1. -1.  1.  1.  1.]
 [ 0.  1. -1. -1. -1.]]
Iteration=2

[[ 1. -1.  1.  1.  1.]
 [ 1.  0.  0.  0.  0.]]
Second phase
Iteration=1

[[-1.  1.  1.  1.]
 [ 2.  0. -1. -2.]]
Iteration=2

[[-1.  1.  1.  1.]
 [ 1.  1.  0. -1.]]
x_res=  [0. 1.]
func(x_res)=  -1.0


Возращаясь к исходной задаче, имеем $ x^{*} = (0, 1)^T, f(x^{*}) = 1 $